Tester for multiple sources

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from core import pds
import numpy as np
from datetime import timedelta

In [5]:
# pds frame definition
pds1d_frame = pds.PDS1D_MultiSource()

In [6]:
# setup mesh
nx = 100
dx = 1
x = np.arange(0, nx * dx, dx)

d = 2
u_init = np.zeros(nx)
t0 = 0
lbc = 'Neumann'
rbc = 'Neumann'

In [7]:
# Load test data
from DSS_analyzer_Mariner import Data1D_GAUGE
test_gauge_dataframe = Data1D_GAUGE.Data1D_GAUGE('testdata/pressure_gauge_data.npz')

from DSS_analyzer_Mariner import Data1D_PumpingCurve
stage4_pumping_curve_dataframe = Data1D_PumpingCurve.Data1D_PumpingCurve('testdata/pumping_curve_stage4.npz')
stage5_pumping_curve_dataframe = Data1D_PumpingCurve.Data1D_PumpingCurve('testdata/pumping_curve_stage5.npz')

In [8]:
# Crop the gauge data into three parts
start_time_stg4 = stage4_pumping_curve_dataframe.get_start_time()
end_time_stg4 = stage4_pumping_curve_dataframe.get_end_time()

start_time_stg5 = stage5_pumping_curve_dataframe.get_start_time()
end_time_stg5 = stage5_pumping_curve_dataframe.get_end_time()

In [9]:
stg4_pg_dataframe = test_gauge_dataframe.copy()
stg4_pg_dataframe.crop(start_time_stg4, end_time_stg4)

In [10]:
interval_pg_dataframe = test_gauge_dataframe.copy()
interval_pg_dataframe.crop(end_time_stg4, start_time_stg5)

In [11]:
stg5_pg_dataframe = test_gauge_dataframe.copy()
stg5_pg_dataframe.crop(start_time_stg5, end_time_stg5)

In [12]:
# pass the parameters
pds1d_frame.set_mesh(x)
pds1d_frame.set_diffusivity(d)
pds1d_frame.set_bcs(lbc, rbc)
pds1d_frame.set_t0(t0)

Diffusivity is a single scalar value, broadcasted to the mesh length.


In [13]:
# Set the source index. This time we have 3 sources. This time will not use finner mesh.
from core import mesh_utils

source_idx_1, x_val = mesh_utils.locate(x, 45)
source_idx_2, x_val = mesh_utils.locate(x, 55)
source_idx_3, x_val = mesh_utils.locate(x, 65)

In [14]:
# pass the parameters to pds1d_frame
pds1d_frame.set_sourceidx([source_idx_1, source_idx_2, source_idx_3])

In [15]:
pds1d_frame.print()

         PDS1D_SingleSource Info         
           mesh: shape=(100,), unique=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
         source: None
            lbc: Neumann
            rbc: Neumann
        initial: None
    diffusivity: shape=(100,), unique=[2]
             t0: 0
          taxis: None
      sourceidx: shape=(3,), unique=[45 55 65]
       snapshot: None
        history: ['Mesh is properly set.', 'Diffusivity is properly set.', 'Boundary conditions are properly set.', 'Initial time set done. \nThe simulation starts at 0.', 'Source index is properly set.']


In [16]:
pds1d_frame.sourceidx.shape

(3,)

TEST -> PASS

In [17]:
# Set sources, I will use the same source for all the sources
total_simulation_time = 1000

In [18]:
# For the test purpose, crop the gauge data
test_gauge_dataframe = stg4_pg_dataframe.copy()
# decide the time range
test_start_time = start_time_stg4 + timedelta(seconds = 200)
test_end_time = test_start_time + timedelta(seconds = total_simulation_time)
test_gauge_dataframe.crop(test_start_time, test_end_time)

In [19]:
test_source = [test_gauge_dataframe, test_gauge_dataframe, test_gauge_dataframe]

In [23]:
pds1d_frame.set_source(test_source)

In [24]:
pds1d_frame.print_log()

Mesh is properly set.
Diffusivity is properly set.
Boundary conditions are properly set.
Initial time set done. 
The simulation starts at 0.
Source index is properly set.
Source term is properly set.


In [25]:
len(pds1d_frame.source)

3

TEST -> PASS

In [26]:
u_init[:] =  test_gauge_dataframe.data[0]
pds1d_frame.set_initial(u_init)

Start the simulation with the initial condition